In [1]:

import requests
import numpy as np 
import pandas as pd
import pickle

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [4]:
rf = pickle.load(open('./models/random_forest.pkl','rb'))
sc = pickle.load(open('./models/standard_scaler.pkl','rb'))
tf = pickle.load(open('./models/tfidf_vectorizer.pkl','rb'))

In [5]:
def search_keyword(text):
    url = "https://rapidapi.p.rapidapi.com/api/search/NewsSearchAPI"
    querystring = {"pageSize":"50",
                   "q": text,
                   "autoCorrect":"true",
                   "pageNumber":"1",
                   "toPublishedDate":"null",
                   "fromPublishedDate":"null"}
    headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "55c35a554dmshab5af4556f598ffp1d3d45jsn8b763429c95f"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    page_1_data=response.json()
    req_df=pd.DataFrame(page_1_data['value'])
    mask=req_df['provider'] == {'name':'wikipedia'}
    #get out of here, wikipedia
    req_df.drop(req_df[mask].index, inplace=True)
    req_df.drop(columns=["id", "isSafe", "image", "keywords"], inplace=True)
    req_df['datePublished']= pd.to_datetime(req_df['datePublished'])
    return req_df.to_html(render_links=True)

In [8]:
def predict_disaster(text_series):
	X = tf.transform(text_series)
	X = pd.DataFrame(X.toarray(), columns = tf.get_feature_names())
	is_disaster = rf.predict(X)
	return is_disaster

In [17]:
x=tf.transform(["this is a test","this is an article about wildfires and hurricanes and stuff"])

In [18]:
x

<2x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [19]:
rf.predict()

array([1, 1], dtype=int64)